# INDIAN RUPEES CURRENCY NOTE DENOMINATION RECOGNITION USING TRANSFER LEARNING WITH TENSORFLOW


---


* This project is a Convolutional Neural Network (CNN) based image classifier model
* Due to CNN models not performing well when built from scratch, this project makes use of feature extraction and fine tuning from pretrained CNNs
* Project developed using Tensorflow Keras
* Dataset sourced from [Kaggle](https://www.kaggle.com/code/kerneler/starter-indian-currency-note-images-f2ab9d26-d/data) 

In [ ]:
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




---


# Dataset Preparation


---


* The dataset does not contain the test images segregated by folders for different classes. So it is not possible to derive the class labels for the test images using Keras image_dataset_from_directory utility
* For this purpose, the images were manually moved to separate folders corresponding to each class of the image

In [ ]:
train_dir = '/content/drive/MyDrive/image_classification/final/training'
test_dir ='/content/drive/MyDrive/image_classification/final/test'
validation_dir = '/content/drive/MyDrive/image_classification/final/validation'

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

tf.random.set_seed(42)

train_data= image_dataset_from_directory(train_dir, label_mode = 'categorical', shuffle=True, seed=42)
test_data = image_dataset_from_directory(test_dir, label_mode = 'categorical', shuffle=True)
validation_data = image_dataset_from_directory(validation_dir, label_mode = 'categorical', shuffle=True)

Found 3582 files belonging to 8 classes.
Found 95 files belonging to 8 classes.
Found 345 files belonging to 8 classes.




---


# Data Augmentation Layer


---


* Can add and experiment around with rescaling, random zoom, random height, etc.
* Rescaling must be avoided for EfficientNetB0

In [ ]:
# Data Augmentation/Preprocessing
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

data_augmentation_layer = keras.Sequential([
    preprocessing.Rescaling(1/255.) #should not be used with efficientnet
])



---


# Transfer Learning


---


* Transfer learning using various pretrained models can be performed for results analysis comparisom

* Model(s) completed till now :
  * EfficientNetB0

## EfficientNetB0


---



### Feature Extraction - Create

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0

tf.random.set_seed(42)
base = EfficientNetB0(include_top = False)
base.trainable = False

inputs = layers.Input(shape=(256,256,3))

#x = data_augmentation_layer(inputs)

x = base(inputs, training = False)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(8, activation="softmax")(x)

efficient_net_b0 = tf.keras.Model(inputs, outputs)

### Feature Extraction - Compile

In [ ]:
# compile
efficient_net_b0.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

### Feature Extraction - Fit

In [ ]:
#fit
history_efficientnetb0 = efficient_net_b0.fit(train_data, epochs = 7 , validation_data=validation_data, validation_steps = int(0.25*len(validation_data)))

Epoch 1/7
112/112 [==============================] - 85s 631ms/step - loss: 1.3790 - accuracy: 0.5723 - val_loss: 1.0000 - val_accuracy: 0.7344
Epoch 2/7
112/112 [==============================] - 70s 588ms/step - loss: 0.8188 - accuracy: 0.7929 - val_loss: 0.7270 - val_accuracy: 0.8125
Epoch 3/7
112/112 [==============================] - 70s 591ms/step - loss: 0.6205 - accuracy: 0.8587 - val_loss: 0.4898 - val_accuracy: 0.9219
Epoch 4/7
112/112 [==============================] - 69s 584ms/step - loss: 0.5098 - accuracy: 0.8900 - val_loss: 0.5681 - val_accuracy: 0.8594
Epoch 5/7
112/112 [==============================] - 70s 586ms/step - loss: 0.4357 - accuracy: 0.9068 - val_loss: 0.4386 - val_accuracy: 0.8906
Epoch 6/7
112/112 [==============================] - 70s 588ms/step - loss: 0.3801 - accuracy: 0.9196 - val_loss: 0.3120 - val_accuracy: 0.9844
Epoch 7/7
112/112 [==============================] - 69s 585ms/step - loss: 0.3390 - accuracy: 0.9299 - val_loss: 0.3782 - val_accuracy:

In [ ]:
efficient_net_b0.evaluate(test_data)

3/3 [==============================] - 3s 363ms/step - loss: 0.3324 - accuracy: 0.9474


[0.33238035440444946, 0.9473684430122375]

### Fine Tuning


In [ ]:
#Last 10 layers are trainable
base.trainable=True

for layers in base.layers[:-5]:
  layers.trainable=False

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/image_classification/final/efficient_net_b0.h5'

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,save_weights_only=True,save_best_only=True)

efficient_net_b0.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                         optimizer=tf.keras.optimizers.Adam(0.0001), 
                        metrics=["accuracy"])

In [ ]:
efficient_net_b0.fit(train_data, epochs = 15, initial_epoch=7, validation_data = validation_data, validation_steps = int(0.15*len(validation_data)),
                     callbacks=[cp_callback])

Epoch 8/15
112/112 [==============================] - 77s 605ms/step - loss: 0.2642 - accuracy: 0.9361 - val_loss: 0.2005 - val_accuracy: 0.9375
Epoch 9/15
112/112 [==============================] - 70s 589ms/step - loss: 0.1695 - accuracy: 0.9604 - val_loss: 0.1433 - val_accuracy: 0.9375
Epoch 10/15
112/112 [==============================] - 69s 584ms/step - loss: 0.1293 - accuracy: 0.9743 - val_loss: 0.2275 - val_accuracy: 0.9062
Epoch 11/15
112/112 [==============================] - 71s 597ms/step - loss: 0.0972 - accuracy: 0.9805 - val_loss: 0.1139 - val_accuracy: 0.9688
Epoch 12/15
112/112 [==============================] - 70s 592ms/step - loss: 0.0783 - accuracy: 0.9886 - val_loss: 0.2841 - val_accuracy: 0.9375
Epoch 13/15
112/112 [==============================] - 71s 599ms/step - loss: 0.0645 - accuracy: 0.9897 - val_loss: 0.0563 - val_accuracy: 0.9688
Epoch 14/15
112/112 [==============================] - 70s 593ms/step - loss: 0.0532 - accuracy: 0.9913 - val_loss: 0.0779 - v

In [ ]:
efficient_net_b0.evaluate(test_data)

3/3 [==============================] - 2s 95ms/step - loss: 0.0313 - accuracy: 1.0000


[0.03130357712507248, 1.0]

In [ ]:
efficient_net_b0.load_weights(checkpoint_dir)
efficient_net_b0.evaluate(test_data)

3/3 [==============================] - 2s 99ms/step - loss: 0.0374 - accuracy: 1.0000


[0.037449274212121964, 1.0]